# K-fold cross validation (k=number of samples =16)

For each run, we test on a single image and train on n_samples-1 images. The final score is in the last cell

In [2]:
from utils.data import Data
from utils.estimators import Dataset, Classifier
import numpy as np
%matplotlib inline

In [4]:
tiff_location = "./Sample/Images/"
shp_location = "./Sample/Labels/"
all_data = Data(tiff_location, shp_location, classes = ["water", "land"])
all_tiff = all_data.read_tiff() 
all_mask = all_data.get_mask()
X, y = all_data.get_Xy(all_tiff, all_mask, n_sample = 200000, k_fold=True)

/home/mibook/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [12]:
classifier = Classifier()
all_preds = []
all_y = []
for i in range(X.shape[0]):
    print(f"Training on split {i+1} out of split {X.shape[0]}")
    _tempX = np.copy(X)
    _tempY = np.copy(y)
    X_test, y_test = _tempX[i], _tempY[i]
    X_train, y_train = np.delete(_tempX, i, 0).reshape(-1, X.shape[2]), np.delete(_tempY, i, 0).reshape(-1, y.shape[2])
    dataset = Dataset(X_train, X_test, y_train, y_test)
    all_y.extend(dataset.testY)
    preds = classifier.random_forest(trainX=dataset.trainX, trainY=dataset.trainY, 
                                     testX=dataset.testX, testY=dataset.testY,
                                     grid_search=False, train=True, 
                                     n_estimators = 10, max_depth = 10)
    all_preds.extend(preds)
all_preds = np.asarray(all_preds)
all_y = np.asarray(all_y)

Training on split 1 out of split 3

Random Forest
Elapsed_time training  0.075389 
Accuracy on train Set: 
1.0
Accuracy on Test Set: 
0.5012
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.00      0.00      5000
           1       0.50      1.00      0.67      5000

    accuracy                           0.50     10000
   macro avg       0.75      0.50      0.34     10000
weighted avg       0.75      0.50      0.34     10000

Confusion Matrix: 
[[  12 4988]
 [   0 5000]]
Training on split 2 out of split 3

Random Forest
Elapsed_time training  0.044353 
Accuracy on train Set: 
1.0
Accuracy on Test Set: 
0.4166
Classification Report: 
              precision    recall  f1-score   support

           0       0.45      0.83      0.59      5000
           1       0.01      0.00      0.00      5000

    accuracy                           0.42     10000
   macro avg       0.23      0.42      0.30     10000
weighted avg       0.23  

/home/mibook/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn import metrics
print("Classification Report: ")
print(metrics.classification_report(all_y, all_preds))
print("Confusion Matrix: ")
print(metrics.confusion_matrix(all_y, all_preds))

Classification Report: 
              precision    recall  f1-score   support

           0       0.46      0.28      0.34     15000
           1       0.48      0.67      0.56     15000

    accuracy                           0.47     30000
   macro avg       0.47      0.47      0.45     30000
weighted avg       0.47      0.47      0.45     30000

Confusion Matrix: 
[[ 4166 10834]
 [ 4988 10012]]
